We need a TFR Dataset with A-values rather than Q-values. ```TournamentData.ipynb``` should provide either another "feature" or another TFR file set for A-values. - DONE

Then RunQVT.ipynb should be used to run the A-values. - DONE

It should be possible to choose the number of games or records in RunQVT. Well, it is, I guess - through the options.

The A-values are derived by subtracting the default value, which can be found beyond the border on a 22x22 board from the q value. - DONE

This should then be divided by 100.0 and the learned A-value should eventually be multiplied by 100, if ever to be re-used in order to recreate the Q-Function. - DONE

The mask is used to not have the beyond-the-border fields influence the loss function. - DONE

HeuristicGomokuPolicy's defense_options are buggy. - DONE

---
## Imitation learning

We'll have a $\pi$-$A$ network consisting of a common residual network with
    - 5-8 layers for line recognition
    - multiple residual blocks
    - residual helps convergence and also reflects the stone-meets-line paradigm
    - network design must allow for an stone-to-field influence across the entire board
        - because the remotest stone could have a role in a threat sequence

Trajectory production is super-slow. Need mass production for imitation training, because the heuristic threat search is slow.

During imitation training, a single combined loss function should help $\pi$ and $A$ network to initialize to efficient networks.

Imitation learning means $\pi$ network learns to predict the moves of the heuristic policy, while the $A$ network learns the advantage from the QFunction that's based on the same heuristics. 

We need the self-play recordings to generate the training data for imitation learning.

Least-significant-move baseline LSMB: May take any other. Has the advantage of creating zeros for all but the relevant fields during the imitation learning phase.


---
## Deep RL
There's only one type of reward: Win or loss. The reward is going to be discounted continuously back to the beginning of the trajectory and provided for $A$-learning.

It's not clear how the different training phases ($\pi$ and $A$) might influence each other. The network might get pushed forth and back.

Maybe it's better to initialize a single network and RL-train $\pi$ and $A$ independently thereafter.

The weights (the critiques) are given by $[r_e - \tilde{A}(s_t)]$, where $r_e$ is episode's final reward and $\tilde{A}$ is the current advantage estimate for the least significant move after $s_t$

# Thinking fast and slow

I want to skip RL in favour of the Alpha-Zero approach with a policy-advised tree search, possibly considering RL for some side-line improvement later. However, I'm still going to initialize the network with Imitation learning from my heuristic policy.

Interestingly, a pretty similar approach has been suggested by [Anthony 2017](https://arxiv.org/pdf/1705.08439.pdf), independent of the research done by Deepmind. Would be interesting to compare the approaches.

[This essay](http://www.moderndescartes.com/essays/deep_dive_mcts/) is the most concise and comprehensible piece on UCT. It refers to a *NeuralNet* to provide value estimates for child nodes. I want to start from that, as it also advises an approach to vectorization to massively improve the performance of the search algorithm. The above algorithm takes a single policy evaluation at the child nodes to estimate a parent's value. It'd be interesting to consider some fast policy to chase down four or five more moves and average their results. Another thing is the formula used for evaluation of the UCB. That's adding 1 to the denominators for stability (precondition for vectorizing the UCB calculation. But it's also omitting the exploration parameter and doesn't take the logarithm on the parent's number of simulations. [This Medium blog](https://medium.com/@quasimik/monte-carlo-tree-search-applied-to-letterpress-34f41c86e238) has the correct formula and some more helpful explanations.

We'll have the architecture derived in [LinesOfFive.ipynb](LinesOfFive.ipynb) learn by imitating the [HeuristicGomokuPolicy](HeuristicPolicy.py). The latter needs to have some function that maps the logic of method ```suggest``` into a learnable distribution.

That should already create a pretty strong player. Additional steps would possibly include ideas from [Anthony 2017](https://arxiv.org/pdf/1705.08439.pdf) to have system 1 (the policy network) and system 2 (the UCT algorithm) learn from each other.

[HeuristicPolicy.ipynb](HeuristicPolicy.ipynb) is now the starting point for creating initial training data. I still need to find out how to effectively reflect the results of the threat sequence search in the resulting action (move) distribution.

I could start with implementing UCT with the heuristic policy and see how it does.

Another hard thing is then the full documentation and operationalization of the entire quest. Providing an interactive interface to play with the algo. A web version of GO-UI also being able to run tournaments. Also, benchmarking my algo against the available players at the official Gomoku tournament site is to be considered.

Last, not least, the entire thing should be presentable on various occasions, meetups, conferences, whatever.